In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# CONFIG
metrics_dir = "./models/results"   
results_dir     = "./models/results/figures"
os.makedirs(results_dir, exist_ok=True)

In [ ]:
# Load all history CSVs, drop 'lr' if present
histories = {}
for fn in sorted(os.listdir(metrics_dir)):
    if fn.lower().endswith(".csv"):
        model = os.path.splitext(fn)[0]
        df = pd.read_csv(os.path.join(metrics_dir, fn), index_col=0)
        if "lr" in df.columns:
            df = df.drop(columns=["lr"])
        histories[model] = df

if not histories:
    raise FileNotFoundError(f"No CSVs found in {metrics_dir}")

In [ ]:
# Plot each metric over epochs
#    Each metric becomes one PNG with one line per model
metrics = list(next(iter(histories.values())).columns)
for metric in metrics:
    plt.figure(figsize=(8,5))
    for model, df in histories.items():
        plt.plot(df.index, df[metric], label=model)
    plt.title(f"{metric.upper()} over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, f"{metric}_curve.png"))
    plt.close()

In [14]:
for metric in metrics:
    plt.figure(figsize=(8,5))

    # find every aug model
    for aug_name in sorted(histories):
        if not aug_name.endswith("_aug"):
            continue
        base = aug_name[:-4]           # strip "_aug"
        orig_name = base               # original has no suffix
        if orig_name not in histories:
            continue

        s_orig = histories[orig_name][metric]
        s_aug  = histories[aug_name][metric]

        # reindex both to same epoch range
        maxlen = max(len(s_orig), len(s_aug))
        idx    = list(range(maxlen))
        s_orig = s_orig.reindex(idx, method="ffill")
        s_aug  = s_aug.reindex(idx, method="ffill")

        diff = s_aug.values - s_orig.values
        plt.plot(idx, diff, label=base)

    plt.axhline(0, color="k", linewidth=1)
    plt.title(f"Δ {metric.upper()} (aug − orig)")
    plt.xlabel("Epoch")
    plt.ylabel(f"Δ {metric}")
    plt.legend(title="model")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, f"{metric}_diff.png"))
    plt.close()

In [15]:
# rows=metrics, cols=models
summary_df = pd.DataFrame({model: df.iloc[-1] for model, df in histories.items()})

# if you had an lr entry in the index, drop it
if "lr" in summary_df.index:
    summary_df = summary_df.drop(index="lr")

summary_df = summary_df.round(2)

In [16]:
clean_csv = os.path.join(results_dir, "metrics_summary_final_epoch_no_lr.csv")
png_path  = os.path.join(results_dir, "metrics_summary_final_epoch_no_lr.png")

In [ ]:
# CONFIG
metrics_dir = "./models/results"
out_csv     = os.path.join(metrics_dir, "metrics_summary_best.csv")
out_png     = os.path.join(metrics_dir, "metrics_summary_best.png")

# gather CSVs
csv_files = [
    fn for fn in os.listdir(metrics_dir)
    if fn.lower().endswith(".csv")
]

best_dfs = []
for fn in csv_files:
    df = pd.read_csv(os.path.join(metrics_dir, fn))
    # pick best epoch by val_loss (or last row if no val_loss)
    if "val_loss" in df:
        best = df.loc[[ df["val_loss"].idxmin() ]].copy()
    else:
        best = df.iloc[[-1]].copy()
    # derive model name
    model_name = os.path.splitext(fn)[0]
    # drop any lr columns
    for col in ("lr", "learning_rate"):
        if col in best:
            best = best.drop(columns=col)
    # round numeric
    best = best.round(2)
    # insert Model
    best.insert(0, "Model", model_name)
    best_dfs.append(best)

# concat
summary = pd.concat(best_dfs, ignore_index=True)

# save CSV of all metrics (optional)
summary.to_csv(out_csv, index=False)
print("Wrote best-epoch metrics →", out_csv)

# select only desired columns for the table
keep = ["Model", "loss", "mae", "val_loss", "val_mae"]
table_df = summary[keep]

# render PNG
n_rows, n_cols = table_df.shape
fig_w = max(6, n_cols * 1.2)
fig_h = max(2, n_rows * 0.4)
fig, ax = plt.subplots(figsize=(fig_w, fig_h))
ax.axis("off")

tbl = ax.table(
    cellText=table_df.values,
    colLabels=[c.replace("_", " ").title() for c in table_df.columns],
    cellLoc="center",
    loc="center"
)
tbl.auto_set_font_size(False)
tbl.set_fontsize(10)
tbl.scale(1, 1.5)

plt.tight_layout()
plt.savefig(out_png, dpi=150, bbox_inches="tight")
plt.close()
print("Wrote filtered table figure →", out_png)


Wrote best-epoch metrics → ./models/results/1\metrics_summary_best.csv
Wrote filtered table figure → ./models/results/1\metrics_summary_best.png
